Batch gradient Descent with early stopping for Soft Max Regression from scratch without using sci-kit learn libraries

Implementing this algorithm on iris dataset

In [4]:
from sklearn.datasets import load_iris
import pandas as pd 
import numpy as np 

C:\Users\saiav\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
iris_data = load_iris()

In [7]:
print(iris_data.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [8]:
#Petal Length and petal width columns for dependent variable and target column as target variable
X = iris_data['data'][:,(2,3)]
y= iris_data['target']

#adding bias to every instance
#Bias is simply a constant value (or a constant vector) that is added to the product of inputs and weights. 
#Bias is utilised to offset the result.

X_bias = np.c_[np.ones([len(X),1]),X]


In [9]:
test_ratio = 0.2
validation_ratio = 0.2
data_size = len(X_bias)
test_size = int(test_ratio*data_size)
validation_size = int(validation_ratio*data_size)
train_size = data_size-test_size-validation_size

random_index = np.random.permutation(data_size)

In [10]:
random_index

array([109, 132, 120,  68,  71, 136,  45,  23, 124,  12,  84,  20,  80,
        25, 127, 102, 121,  18,  36, 112,  94, 115,  70,  64, 119,   5,
        89,  48,  11,  85, 114,  37,  42,  51,  61,  28,  32,  81,  38,
       133,  75,   8, 107, 116, 106, 130, 110,  49, 128, 122,  19,   4,
        27,  35, 142,  31, 147,  98,  74,  59,  41,   9,  50,  87, 134,
       144,  13, 135, 137, 108,  90,  58,  24,  77, 129, 113,  17,  83,
        47, 146,  44, 101,  91, 100, 148,   3,  72, 138,  22,  33, 149,
        29,  92,  73,  55, 143, 141, 103, 123,  15,  82, 104,  26,   6,
        67,  88, 145,  56,  10,  69, 131,  76,   2,   0,  39,  14,  62,
       105,  16,  95,  65,  78,   1,  30,  21,  79,  97,  99, 140,  52,
        40,  66,  34,  93,  60,  86, 118,  96,  53,  54, 139, 126,  63,
       125,   7, 111,  43,  57, 117,  46])

In [12]:
X_train = X_bias[random_index[:train_size]]
X_test = X_bias[random_index[-test_size:]]
y_train = y[random_index[:train_size]]
y_test = y[random_index[-test_size:]]
X_valid = X_bias[random_index[train_size:-test_size]]
y_valid = y[[random_index[train_size:-test_size]]]

C:\Users\saiav\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [14]:
random_index[train_size:-test_size]

array([149,  29,  92,  73,  55, 143, 141, 103, 123,  15,  82, 104,  26,
         6,  67,  88, 145,  56,  10,  69, 131,  76,   2,   0,  39,  14,
        62, 105,  16,  95])

In [21]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [29]:
def one_hot_encode(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [35]:
one_hot_encode(y_train[:10])

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [36]:
y_test_onehot = one_hot_encode(y_test)
y_train_onehot = one_hot_encode(y_train)
y_valid_one = one_hot_encode(y_valid)

Now let's implement the Softmax function. Equation for softmax function:

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [37]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps,axis = 1, keepdims=True)
    return exps/exp_sums

In [41]:
inputs_number = X_train.shape[1]
outputs_number = len(np.unique(y_train))